# Sentimental Analysis Model

In [2]:
# Importing  dependencies
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from gensim.models import Word2Vec
import pandas as pd
import nltk
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
import numpy as np


In [3]:
# Reading CSV file
df = pd.read_csv('Resources/bitcointweets.csv',header=None)
df.columns =["DateTime","Tweet","Handle","Followers","Unknown","Hashtags","Ref", "Sentiment"]
df.head()

,DateTime,Tweet,Handle,Followers,Unknown,Hashtags,Ref,Sentiment
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,16522,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral']
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,1295,0,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral']
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,6090,0,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive']
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,2626,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive']
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,184,0,[],"<a href=""http://twitter.com/download/android"" ...",['positive']


In [4]:
# Replace Sentiment Column list into strings of categorical variables
df['Sentiment'] = df['Sentiment'].replace(
    {"['positive']": 'positive', "['negative']": 'negative',"['neutral']": 'neutral'})
df.head()

,DateTime,Tweet,Handle,Followers,Unknown,Hashtags,Ref,Sentiment
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,16522,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",neutral
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,1295,0,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",neutral
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,6090,0,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",positive
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,2626,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",positive
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,184,0,[],"<a href=""http://twitter.com/download/android"" ...",positive


In [5]:
# Using sklearn to encode categorical variables
from sklearn import preprocessing
encoder= preprocessing.LabelEncoder()
categories=["positive","negative","neutral"]
encoder.fit(categories)
#new_categories=["negative","positive", "positive","neutral"]
#encoder.transform(new_categories)

LabelEncoder()

# Categorical Labels: Positive=2, Negative=0, Neutral=1

In [6]:
# Using encoder to transform Sentiment column
df['Sentiment']=encoder.transform(df['Sentiment'])
df.head()
#df['Sentiment'].unique()

,DateTime,Tweet,Handle,Followers,Unknown,Hashtags,Ref,Sentiment
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,16522,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",1
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,1295,0,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",1
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,6090,0,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,2626,0,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",2
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,184,0,[],"<a href=""http://twitter.com/download/android"" ...",2


In [7]:
# Use train_test_split to create training and testing data
from sklearn.model_selection import train_test_split
X= df['Tweet']
y= df['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y)


In [8]:
# This function converts a text to a sequence of words
def review_wordlist(review, remove_stopwords=False):
    # 1. Removing html tags
    review_text = BeautifulSoup(review).get_text()
    # 2. Removing non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # 3. Converting to lower case and splitting
    words = review_text.lower().split()
    # 4. Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return(words)

In [9]:
# word2vec expects list of lists
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\13477\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [10]:
# word2vec expects a list of lists.
# Using punkt tokenizer for better splitting of a paragraph into sentences.

import nltk.data
nltk.download('popular')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\13477\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\13477\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\13477\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\13477\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\13477\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]   

In [11]:
# This function splits a review into sentences
def review_sentences(review, tokenizer, remove_stopwords=False):
    # 1. Using nltk tokenizer
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # 2. Loop for each sentence
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    # This returns the list of lists
    return sentences

In [12]:
# Parsing sentences from the training set

sentences = []
print("Parsing sentences from training set")
for review in X_train:
    sentences += review_sentences(review, tokenizer)

Parsing sentences from training set


C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/gK5NmZFtAN" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/k5FuSeoF0z" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/qMW8fh42FV" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/ctxvgr8cpQ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/WYhr2kkhRA" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/pu6fGrco8W" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/JUxO0oSPaN" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/TvryoMvjkH" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/JZRkFOUVqE" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/xbta0IVJJ9" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/KVUKtQavkm…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/fQTE1VSnKJ" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

...

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/dQkVD1jSTp" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/EHEERm1b4y" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/PwrTvySYdh" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/z5uoSPmihB" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/lZGNHXj…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/0iOKopYKqA" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind t

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/e66rnvFKFM" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/35IojAXFPp" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/cIAaRMfiWc" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/X2hHn0…" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/xSDZJiQSvv" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/gySD4DYkC6" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind th

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/dPXrATL78C" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/QmAeAEAyh5" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:335: UserWarning: "https://t.co/ARnjisAS9u" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind

In [13]:
# Importing the built-in logging module
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [14]:
# Creating the model and setting values for the various parameters
num_features = 300  # Word vector dimensionality
min_word_count = 40 # Minimum word count
num_workers = 4     # Number of parallel threads
context = 10        # Context window size
downsampling = 1e-3 # (0.001) Downsample setting for frequent words

# Initializing the train model
from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling)

# To make the model memory efficient
model.init_sims(replace=True)

# Saving the model for later use. Can be loaded using Word2Vec.load()
model_name = "300features_40minwords_10context.h5"
model.save(model_name)

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
2019-07-08 18:58:33,137 : INFO : collecting all words and their counts
2019-07-08 18:58:33,137 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-08 18:58:33,163 : INFO : PROGRESS: at sentence #10000, processed 116376 words, keeping 12754 word types
2019-07-08 18:58:33,186 : INFO : PROGRESS: at sentence #20000, processed 233503 words, keeping 20151 word types
2019-07-08 18:58:33,213 : INFO : PROGRESS: at sentence #30000, processed 349850 words, keeping 26625 word types
2019-07-08 18:58:33,236 : INFO : PROGRESS: at sentence #40000, processed 466816 words, keeping 32451 word types
2019-07-08 18:58:33,262 : INFO : PROGRESS: at sentence #50000, processed 584848 words, keeping 37709 word t

Training model....


2019-07-08 18:58:33,298 : INFO : collected 45110 word types from a corpus of 755201 raw words and 64631 sentences
2019-07-08 18:58:33,299 : INFO : Loading a fresh vocabulary
2019-07-08 18:58:33,315 : INFO : effective_min_count=40 retains 1589 unique words (3% of original 45110, drops 43521)
2019-07-08 18:58:33,316 : INFO : effective_min_count=40 leaves 637139 word corpus (84% of original 755201, drops 118062)
2019-07-08 18:58:33,322 : INFO : deleting the raw counts dictionary of 45110 items
2019-07-08 18:58:33,324 : INFO : sample=0.001 downsamples 47 most-common words
2019-07-08 18:58:33,326 : INFO : downsampling leaves estimated 433495 word corpus (68.0% of prior 637139)
2019-07-08 18:58:33,330 : INFO : estimated required memory for 1589 words and 300 dimensions: 4608100 bytes
2019-07-08 18:58:33,330 : INFO : resetting layer weights
2019-07-08 18:58:33,352 : INFO : training model with 4 workers on 1589 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2019

2019-07-08 19:03:15,908 : INFO : EPOCH 2 - PROGRESS: at 81.95% examples, 2642 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:03:18,106 : INFO : EPOCH 2 - PROGRESS: at 86.02% examples, 2725 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:03:24,288 : INFO : EPOCH 2 - PROGRESS: at 87.39% examples, 2646 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:03:26,321 : INFO : EPOCH 2 - PROGRESS: at 91.35% examples, 2729 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:03:32,708 : INFO : EPOCH 2 - PROGRESS: at 92.65% examples, 2651 words/s, in_qsize 6, out_qsize 0
2019-07-08 19:03:34,425 : INFO : EPOCH 2 - PROGRESS: at 96.60% examples, 2733 words/s, in_qsize 3, out_qsize 1
2019-07-08 19:03:34,426 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-08 19:03:37,605 : INFO : EPOCH 2 - PROGRESS: at 97.37% examples, 2698 words/s, in_qsize 2, out_qsize 1
2019-07-08 19:03:37,607 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-08 19:03:38,962 : INFO : EPOCH 2 

2019-07-08 19:06:50,375 : INFO : EPOCH 4 - PROGRESS: at 15.95% examples, 2591 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:06:57,027 : INFO : EPOCH 4 - PROGRESS: at 17.30% examples, 2246 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:06:58,195 : INFO : EPOCH 4 - PROGRESS: at 19.94% examples, 2503 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:07:00,049 : INFO : EPOCH 4 - PROGRESS: at 21.24% examples, 2533 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:07:04,132 : INFO : EPOCH 4 - PROGRESS: at 22.57% examples, 2418 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:07:07,058 : INFO : EPOCH 4 - PROGRESS: at 23.88% examples, 2385 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:07:09,625 : INFO : EPOCH 4 - PROGRESS: at 26.53% examples, 2501 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:07:12,500 : INFO : EPOCH 4 - PROGRESS: at 27.82% examples, 2471 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:07:15,701 : INFO : EPOCH 4 - PROGRESS: at 29.15% examples, 2430 words/s, in_qsize 7, out_qsize 0
2

2019-07-08 19:09:58,471 : INFO : EPOCH 5 - PROGRESS: at 31.78% examples, 2620 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:10:01,500 : INFO : EPOCH 5 - PROGRESS: at 33.11% examples, 2580 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:10:04,473 : INFO : EPOCH 5 - PROGRESS: at 34.46% examples, 2548 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:10:06,308 : INFO : EPOCH 5 - PROGRESS: at 35.85% examples, 2566 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:10:07,798 : INFO : EPOCH 5 - PROGRESS: at 37.13% examples, 2596 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:10:10,189 : INFO : EPOCH 5 - PROGRESS: at 38.46% examples, 2588 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:10:12,777 : INFO : EPOCH 5 - PROGRESS: at 39.81% examples, 2573 words/s, in_qsize 8, out_qsize 0
2019-07-08 19:10:14,911 : INFO : EPOCH 5 - PROGRESS: at 41.08% examples, 2577 words/s, in_qsize 7, out_qsize 0
2019-07-08 19:10:16,448 : INFO : EPOCH 5 - PROGRESS: at 42.38% examples, 2600 words/s, in_qsize 7, out_qsize 0
2

In [15]:
# This will give the total number of words in the vocabolary created from this dataset
model.wv.syn0.shape

C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


(1589, 300)

In [16]:
# Function to average all word vectors in a paragraph
def featureVecMethod(words, model, num_features):
    # Pre-initialising empty numpy array for speed
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    #Converting Index2Word which is a list to a set for better speed in the execution.
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Dividing the result by number of words to get average
    featureVec = np.divide(featureVec, nwords)
    return featureVec

In [17]:
# Function for calculating the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        # Printing a status message every 1000th review
        if counter%1000 == 0:
            print("Review %d of %d"%(counter,len(reviews)))
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs

In [18]:
# Calculating average feature vector for training set
clean_train_reviews = []
for review in X_train:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

Review 0 of 38144
Review 1000 of 38144
Review 2000 of 38144


C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 3000 of 38144
Review 4000 of 38144
Review 5000 of 38144
Review 6000 of 38144
Review 7000 of 38144
Review 8000 of 38144
Review 9000 of 38144
Review 10000 of 38144
Review 11000 of 38144
Review 12000 of 38144
Review 13000 of 38144
Review 14000 of 38144
Review 15000 of 38144
Review 16000 of 38144
Review 17000 of 38144
Review 18000 of 38144
Review 19000 of 38144
Review 20000 of 38144
Review 21000 of 38144
Review 22000 of 38144
Review 23000 of 38144
Review 24000 of 38144
Review 25000 of 38144
Review 26000 of 38144
Review 27000 of 38144
Review 28000 of 38144
Review 29000 of 38144
Review 30000 of 38144
Review 31000 of 38144
Review 32000 of 38144
Review 33000 of 38144
Review 34000 of 38144
Review 35000 of 38144
Review 36000 of 38144
Review 37000 of 38144
Review 38000 of 38144


In [19]:
# Calculating average feature vactors for test set     
clean_test_reviews = []
for review in X_test:
    clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

Review 0 of 12715
Review 1000 of 12715


C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 2000 of 12715
Review 3000 of 12715
Review 4000 of 12715
Review 5000 of 12715
Review 6000 of 12715
Review 7000 of 12715
Review 8000 of 12715
Review 9000 of 12715
Review 10000 of 12715
Review 11000 of 12715
Review 12000 of 12715


In [20]:
# Determining shape of vectors for train data
trainDataVecs.shape

(38144, 300)

In [21]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
    
print("Fitting random forest to training data....")    
forest = forest.fit(trainDataVecs, y_train)
forest

Fitting random forest to training data....


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
# Determining our score on training data 
forest.score(trainDataVecs, y_train)

0.9970637583892618

In [23]:
# Determining our score on testing data 
forest.score(testDataVecs, y_test)

0.8993314982304365

In [24]:
# Predicting the sentiment values for test data and saving the results in a csv file 
result = forest.predict(testDataVecs)
result

array([0, 1, 1, ..., 2, 0, 1])

In [25]:
# Setting the output file wuth predicted sentiments and saving it as a csv data
output = pd.DataFrame(data={"Tweet":X_test,"sentiment":result})
output.to_csv("Output/output.csv", index=False, header=True)

# Generating Live Tweet data

In [27]:
import tweepy as tw
import pandas as pd
import json


In [ ]:
## Twitter credentials
with open("./secrets.json") as f:
    secrets = json.load(f)
consumer_key = secrets["twconsumer_key"]
consumer_secret = secrets["twconsumer_secret"]
access_token = secrets["twaccess_token"]
access_token_secret = secrets["twaccess_secret"]

In [ ]:
#Authenticate
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
# Define the search term and the date_since date as variables
search_words = "#bitcoin" + "-filter:retweets"
date_since = "2019-07-01"

In [ ]:
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(500)
tweets

In [ ]:
# provides the user’s twitter handle 
# provides the user’s provided location

tweets = tw.Cursor(api.search, 
                           q=search_words,
                           lang="en",
                           since=date_since).items(500)


users_locs = [[tweet.created_at, tweet.text, tweet.user.screen_name, tweet.user.followers_count, tweet.user.location, tweet.retweet_count] for tweet in tweets]
users_locs

In [ ]:
tweet_text = pd.DataFrame(data=users_locs, 
                    columns=["DateTime","Tweet","Handle","Followers","Location","Retweet Count"])
tweet_text

# Running model on the live tweet data

In [44]:
# Currently all three days twitter dumps are provided in the Dump folder
df_07_06_2019 = pd.read_csv('Dumps/twitterdump_07.06.2019.csv')
df_07_07_2019 = pd.read_csv('Dumps/twitterdump_07.07.2019.csv')
df_07_08_2019 = pd.read_csv('Dumps/twitterdump_07.08.2019.csv')

#df.columns =["DateTime","Tweet","Handle","Followers","Unknown","Hashtags","Ref", "Sentiment"]
df_07_08_2019.head()

,DateTime,Tweet,Handle,Followers,Location,Retweet Count
0,2019-07-08 15:46:44,Cryptocurrency skeptics who only focus on Bitc...,EuropeFcoin,626,NaN,0
1,2019-07-08 15:46:43,Bitcoin mining on an Apollo Guidance Computer:...,monitor_bitcoin,187,NaN,0
2,2019-07-08 15:46:43,Stepan Snigirev: “Lightning secrets – nobody w...,WorldCryptoNet,62103,The World,0
3,2019-07-08 15:46:42,The Spanish Prisoner: On the Future of Libra a...,monitor_bitcoin,187,NaN,0
4,2019-07-08 15:46:42,Alekos Filini: “One plugin (in c-lightning) a ...,WorldCryptoNet,62103,The World,0


In [45]:
frames = [df_07_06_2019,df_07_07_2019,df_07_08_2019]
tweet_text=pd.concat(frames)

tweet_text


,DateTime,Tweet,Handle,Followers,Location,Retweet Count
0,2019-07-06 17:23:51,$BCH and $BSV #hodler... smh \n🤣🤣🤣🤣\n\n#crypto...,KoalaCryptos,564,The Moon,0
1,2019-07-06 17:23:43,LAST TRADE: SELL 0.03562599BTC@10287.1EUR\nSEL...,digital_mine_,5750,STEEM BLOCKCHAIN,0
2,2019-07-06 17:23:05,With this project that consists of a good team...,maruf07388605,734,Bangladesh,0
3,2019-07-06 17:23:02,Join the faucet hub for free #bitcoin #cryptoc...,BitcoinTap,270,Tokyo,0
4,2019-07-06 17:22:55,"Roses are red,\nViolets are blue,\nAnd Craig W...",Coin_Brawl,148,NaN,0
5,2019-07-06 17:22:40,@Citi This comes up in Crypto twitter. Keep tr...,BrianSchweitz11,12,NaN,0
6,2019-07-06 17:22:04,Looks like an inverted head &amp; shoulders pa...,KevinZimmer8,16,"West Bend, WI",0
7,2019-07-06 17:21:47,Bitcoin Price Prediction And Analysis For June...,Remi_Vladuceanu,161179,Blockchain,0
8,2019-07-06 17:21:25,The irony of #Bitcoin: a decentralized currenc...,NeeshaRemak,4,"Brooklyn, NY",0
9,2019-07-06 17:21:12,Update: I've invested all of the savings in #b...,thedridge,1792,Santa Clara,0


In [46]:

# Calculating average feature vactors for test set     
clean_test_live_reviews = []
for review in tweet_text['Tweet']:
    clean_test_live_reviews.append(review_wordlist(review,remove_stopwords=True))
    
testDataVecs_live = getAvgFeatureVecs(clean_test_live_reviews, model, num_features)

Review 0 of 3000
Review 1000 of 3000
Review 2000 of 3000


C:\Users\13477\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [47]:
# Predicting the sentiment values for test data and saving the results in a csv file 
result = forest.predict(testDataVecs_live)
#result

In [48]:
# Collecting the predicted sentiments and storing it as an output dataframe
output_live = pd.DataFrame(data={"DateTime": tweet_text['DateTime'],"Tweet":tweet_text['Tweet'],"Handle":tweet_text['Handle'],"Followers":tweet_text['Followers'],"Location":tweet_text['Location'],"Retweet Count":tweet_text['Retweet Count'],"Sentiment":result})
output_live.head()

,DateTime,Tweet,Handle,Followers,Location,Retweet Count,Sentiment
0,2019-07-06 17:23:51,$BCH and $BSV #hodler... smh \n🤣🤣🤣🤣\n\n#crypto...,KoalaCryptos,564,The Moon,0,1
1,2019-07-06 17:23:43,LAST TRADE: SELL 0.03562599BTC@10287.1EUR\nSEL...,digital_mine_,5750,STEEM BLOCKCHAIN,0,1
2,2019-07-06 17:23:05,With this project that consists of a good team...,maruf07388605,734,Bangladesh,0,2
3,2019-07-06 17:23:02,Join the faucet hub for free #bitcoin #cryptoc...,BitcoinTap,270,Tokyo,0,2
4,2019-07-06 17:22:55,"Roses are red,\nViolets are blue,\nAnd Craig W...",Coin_Brawl,148,NaN,0,1


In [49]:
# write to csv file
output_live.to_csv("Output/live_output.csv",index= False, header=True)

In [ ]:
import pandas as pd
data = [{'name': 'vikash', 'age': 27}, {'name': 'Satyam', 'age': 14}]
df = pd.DataFrame.from_dict(data, orient='columns')
